In [2]:
pip! install numpy

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "!"



In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.naive_bayes import GaussianNB,ComplementNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.neighbors import KNeighborsClassifier,LocalOutlierFactor
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,accuracy_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
import re

ModuleNotFoundError: No module named 'numpy'

In [2]:
df=pd.read_csv(r'C:\Users\Bhavya\Desktop\HateSpeechDataset\labeled_data.csv')
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
def remove_and_extract_rt_username(text):
    rt_pattern = re.search(r'RT\s+@[\w]+:', text)
    if rt_pattern:
        extracted_pattern = rt_pattern.group()
        text_without_rt = text.replace(extracted_pattern, '')
        return text_without_rt, extracted_pattern
    else:
        return text, None

def extract_hashtags(text):
    hashtags = re.findall(r'#\w+', text)
    return hashtags

def remove_non_ascii(text):
    return text.encode('ascii', 'ignore').decode()

def extract_urls(text):
    urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    return urls

def remove_urls(text):
    return re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

def remove_punctuation(s):
    s = re.sub(r'[^\w\s]', '', s)
    return s

def remove_stopwords(s):
    s = ' '.join(word for word in s.split() if word not in stop_words)
    return s

def remove_symbols(s):
    s = re.sub(r'[^a-zA-Z\s]', '', s)
    return s

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def stem_text(s):
    s = ' '.join(stemmer.stem(word) for word in s.split())
    return s
def lemmatize_text(s):
    s = ' '.join(lemmatizer.lemmatize(word) for word in s.split())
    return s


In [6]:
!pip install nltk

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bhavya\AppData\Roaming\nltk_data...


True

In [8]:
df['Hashtags'] = df['tweet'].apply(extract_hashtags)
df['text_without_hashtag'] = df['tweet'].apply(lambda x: re.sub(r'#\w+', '', x))
df['text_without_hashtag_rt'], df['rt_username'] = zip(*df['text_without_hashtag'].apply(remove_and_extract_rt_username))
df['urls'] = df['text_without_hashtag_rt'].apply(extract_urls)
df['text_without_urls'] = df['text_without_hashtag_rt'].apply(remove_urls)
df['cleaned_text'] = df['text_without_hashtag_rt'].apply(remove_urls)
df['cleaned_text2']=df['cleaned_text'].str.replace('[^\w\s]','')
df['cleaned_text3']=df['cleaned_text2'].apply(str.lower)
df['cleaned_text4']=df['cleaned_text3'].apply(remove_symbols)
df['stemmed']=df['cleaned_text4'].apply(stem_text)
df['lemmatized']=df['cleaned_text4'].apply(lemmatize_text)

In [9]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,Hashtags,text_without_hashtag,text_without_hashtag_rt,rt_username,urls,text_without_urls,cleaned_text,cleaned_text2,cleaned_text3,cleaned_text4,stemmed,lemmatized
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,[],!!! RT @mayasolovely: As a woman you shouldn't...,!!! As a woman you shouldn't complain about c...,RT @mayasolovely:,[],!!! As a woman you shouldn't complain about c...,!!! As a woman you shouldn't complain about c...,!!! As a woman you shouldn't complain about c...,!!! as a woman you shouldn't complain about c...,as a woman you shouldnt complain about clean...,as a woman you shouldnt complain about clean u...,a a woman you shouldnt complain about cleaning...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,[],!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,!!!!! boy dats cold...tyga dwn bad for cuffin...,RT @mleew17:,[],!!!!! boy dats cold...tyga dwn bad for cuffin...,!!!!! boy dats cold...tyga dwn bad for cuffin...,!!!!! boy dats cold...tyga dwn bad for cuffin...,!!!!! boy dats cold...tyga dwn bad for cuffin...,boy dats coldtyga dwn bad for cuffin dat hoe...,boy dat coldtyga dwn bad for cuffin dat hoe in...,boy dat coldtyga dwn bad for cuffin dat hoe in...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,[],!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,!!!!!!! RT @UrKindOfBrand Dawg!!!! You ever f...,RT @80sbaby4life:,[],!!!!!!! RT @UrKindOfBrand Dawg!!!! You ever f...,!!!!!!! RT @UrKindOfBrand Dawg!!!! You ever f...,!!!!!!! RT @UrKindOfBrand Dawg!!!! You ever f...,!!!!!!! rt @urkindofbrand dawg!!!! you ever f...,rt urkindofbrand dawg you ever fuck a bitch ...,rt urkindofbrand dawg you ever fuck a bitch an...,rt urkindofbrand dawg you ever fuck a bitch an...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,[],!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,!!!!!!!!! @viva_based she look like a tranny,RT @C_G_Anderson:,[],!!!!!!!!! @viva_based she look like a tranny,!!!!!!!!! @viva_based she look like a tranny,!!!!!!!!! @viva_based she look like a tranny,!!!!!!!!! @viva_based she look like a tranny,vivabased she look like a tranny,vivabas she look like a tranni,vivabased she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,[#57361],!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,!!!!!!!!!!!!! The shit you hear about me migh...,RT @ShenikaRoberts:,[],!!!!!!!!!!!!! The shit you hear about me migh...,!!!!!!!!!!!!! The shit you hear about me migh...,!!!!!!!!!!!!! The shit you hear about me migh...,!!!!!!!!!!!!! the shit you hear about me migh...,the shit you hear about me might be true or ...,the shit you hear about me might be true or it...,the shit you hear about me might be true or it...


In [10]:
x=df['cleaned_text4'].values
vectorizer = CountVectorizer()  
x = vectorizer.fit_transform(x)
y=df['class']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=51,stratify=y)
models = dict()
results=dict()
models['Logistic Regression'] = LogisticRegression(max_iter=5000)
models['Multinomial Naive Bayes'] = MultinomialNB()
models['Complement Naive Bayes'] = ComplementNB(force_alpha=True)
models['KMeans'] = KMeans(n_clusters=2, n_init=10, random_state=59)
models['XGB']= XGBClassifier(n_estimators=500)
models['Support Vector Machine'] = SVC(kernel = 'sigmoid', gamma='scale',probability=True)
models['Decision Tree'] = DecisionTreeClassifier(max_depth=100)
models['kNN'] = KNeighborsClassifier()
models['SGD']=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=100, tol=None)
models['adaboost']=AdaBoostClassifier()
models['Random Forest'] = RandomForestClassifier(n_estimators=100)
print("Training Models:\n")
for model in models:
        models[model].fit(x_train,y_train)
        print("model "+str(model)+" trained")
print("Test Set Prediction:\n")
model_names,accuracies,precisions_w,recalls_w,f1_scores_w,precisions_m,recalls_m,f1_scores_m = [],[],[],[],[],[],[],[]
for i in models:
        print(i)
        y_pred=models[i].predict(x_test)
        print(classification_report(y_test,y_pred))
        accuracy = accuracy_score(y_test, y_pred)
        if i!='SGD':
                results[models[i]]=accuracy
        precision_w = precision_score(y_test, y_pred,average='weighted')
        recall_w = recall_score(y_test, y_pred,average='weighted')
        f1_w = f1_score(y_test, y_pred,average='weighted')
        precision_m = precision_score(y_test, y_pred,average='macro')
        recall_m = recall_score(y_test, y_pred,average='macro')
        f1_m = f1_score(y_test, y_pred,average='macro')
        model_names.append(i)
        accuracies.append(accuracy)
        precisions_w.append(precision_w)
        recalls_w.append(recall_w)
        f1_scores_w.append(f1_w)
        precisions_m.append(precision_m)
        recalls_m.append(recall_m)
        f1_scores_m.append(f1_m)
top_3_keys_bow = sorted(results, key=lambda k: results[k], reverse=True)[:3]
c1=top_3_keys_bow[0]
c2=top_3_keys_bow[1]
c3=top_3_keys_bow[2]
models['ensemble voting'] = VotingClassifier (estimators=[('clf1',c1), ('clf2',c2),('clf3',c3)], voting='soft')
models['ensemble voting'].fit(x_train, y_train)
y_pred = models['ensemble voting'].predict(x_test)
print('-'*20+'VotingClassifier'+'-'*20)
print(classification_report(y_test, y_pred,digits=5))
cf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision_w = precision_score(y_test, y_pred,average='weighted')
recall_w = recall_score(y_test, y_pred,average='weighted')
f1_w = f1_score(y_test, y_pred,average='weighted')
precision_m = precision_score(y_test, y_pred,average='macro')
recall_m = recall_score(y_test, y_pred,average='macro')
f1_m = f1_score(y_test, y_pred,average='macro')
model_names.append('ensemble voting')
accuracies.append(accuracy)
precisions_w.append(precision_w)
recalls_w.append(recall_w)
f1_scores_w.append(f1_w)
precisions_m.append(precision_m)
recalls_m.append(recall_m)
f1_scores_m.append(f1_m)
metrics_df = pd.DataFrame({
    'Model Name': model_names,
    'Accuracy': accuracies,
    'Precision Weighted': precisions_w,
    'Recall Weighted': recalls_w,
    'F1-Score Weighted': f1_scores_w,
    'Precision Macro': precisions_m,
    'Recall Macro': recalls_m,
    'F1-Score Macro': f1_scores_m
})
metrics_df.sort_values(by=['Accuracy'], ascending=False, inplace=True)
metrics_df.to_csv("Bow.csv")

Training Models:

model Logistic Regression trained
model Multinomial Naive Bayes trained
model Complement Naive Bayes trained
model KMeans trained


C:\Users\Bhavya\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


model XGB trained
model Support Vector Machine trained
model Decision Tree trained
model kNN trained
model SGD trained


c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


model adaboost trained
model Random Forest trained
Test Set Prediction:

Logistic Regression
              precision    recall  f1-score   support

           0       0.49      0.25      0.33       429
           1       0.93      0.95      0.94      5757
           2       0.84      0.86      0.85      1249

    accuracy                           0.90      7435
   macro avg       0.75      0.69      0.71      7435
weighted avg       0.89      0.90      0.89      7435

Multinomial Naive Bayes
              precision    recall  f1-score   support

           0       0.33      0.07      0.12       429
           1       0.87      0.97      0.92      5757
           2       0.85      0.59      0.69      1249

    accuracy                           0.86      7435
   macro avg       0.68      0.55      0.58      7435
weighted avg       0.83      0.86      0.83      7435

Complement Naive Bayes
              precision    recall  f1-score   support

           0       0.33      0.27      0.30

c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

              precision    recall  f1-score   support

           0       0.52      0.23      0.32       429
           1       0.93      0.95      0.94      5757
           2       0.83      0.90      0.86      1249

    accuracy                           0.90      7435
   macro avg       0.76      0.70      0.71      7435
weighted avg       0.89      0.90      0.89      7435

Support Vector Machine
              precision    recall  f1-score   support

           0       0.35      0.20      0.26       429
           1       0.92      0.94      0.93      5757
           2       0.78      0.81      0.79      1249

    accuracy                           0.87      7435
   macro avg       0.68      0.65      0.66      7435
weighted avg       0.86      0.87      0.87      7435

Decision Tree
              precision    recall  f1-score   support

           0       0.37      0.33      0.35       429
           1       0.94      0.93      0.93      5757
           2       0.81      0.88     

c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


--------------------VotingClassifier--------------------
              precision    recall  f1-score   support

           0    0.50265   0.22145   0.30744       429
           1    0.92948   0.95241   0.94080      5757
           2    0.82999   0.89512   0.86133      1249

    accuracy                        0.90061      7435
   macro avg    0.75404   0.68966   0.70319      7435
weighted avg    0.88814   0.90061   0.89091      7435



In [11]:
x=df['cleaned_text4'].values
vectorizer = TfidfVectorizer()  
x = vectorizer.fit_transform(x)
y=df['class']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=51,stratify=y)
models = dict()
results=dict()
models['Linear Regression'] = LogisticRegression(max_iter=5000)
models['Multinomial Naive Bayes'] = MultinomialNB()
models['Complement Naive Bayes'] = ComplementNB(force_alpha=True)
models['KMeans'] = KMeans(n_clusters=2, n_init=10, random_state=59)
models['XGB']= XGBClassifier(n_estimators=500)
models['Support Vector Machine'] = SVC(kernel = 'sigmoid', gamma='scale',probability=True)
models['Decision Tree'] = DecisionTreeClassifier(max_depth=100)
models['kNN'] = KNeighborsClassifier()
models['SGD']=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=100, tol=None)
models['adaboost']=AdaBoostClassifier()
models['Random Forest'] = RandomForestClassifier(n_estimators=100)
print("Training Models:\n")
for model in models:
        models[model].fit(x_train,y_train)
        print("model "+str(model)+" trained")
print("Test Set Prediction:\n")
model_names,accuracies,precisions_w,recalls_w,f1_scores_w,precisions_m,recalls_m,f1_scores_m = [],[],[],[],[],[],[],[]
for i in models:
        print(i)
        y_pred=models[i].predict(x_test)
        print(classification_report(y_test,y_pred))
        accuracy = accuracy_score(y_test, y_pred)
        if i!='SGD':
                results[models[i]]=accuracy
        precision_w = precision_score(y_test, y_pred,average='weighted')
        recall_w = recall_score(y_test, y_pred,average='weighted')
        f1_w = f1_score(y_test, y_pred,average='weighted')
        precision_m = precision_score(y_test, y_pred,average='macro')
        recall_m = recall_score(y_test, y_pred,average='macro')
        f1_m = f1_score(y_test, y_pred,average='macro')
        model_names.append(i)
        accuracies.append(accuracy)
        precisions_w.append(precision_w)
        recalls_w.append(recall_w)
        f1_scores_w.append(f1_w)
        precisions_m.append(precision_m)
        recalls_m.append(recall_m)
        f1_scores_m.append(f1_m)
top_3_keys_bow = sorted(results, key=lambda k: results[k], reverse=True)[:3]
c1=top_3_keys_bow[0]
c2=top_3_keys_bow[1]
c3=top_3_keys_bow[2]
models['ensemble voting'] = VotingClassifier (estimators=[('clf1',c1), ('clf2',c2),('clf3',c3)], voting='soft')
models['ensemble voting'].fit(x_train, y_train)
y_pred = models['ensemble voting'].predict(x_test)
print('-'*20+'VotingClassifier'+'-'*20)
print(classification_report(y_test, y_pred,digits=5))
cf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision_w = precision_score(y_test, y_pred,average='weighted')
recall_w = recall_score(y_test, y_pred,average='weighted')
f1_w = f1_score(y_test, y_pred,average='weighted')
precision_m = precision_score(y_test, y_pred,average='macro')
recall_m = recall_score(y_test, y_pred,average='macro')
f1_m = f1_score(y_test, y_pred,average='macro')
model_names.append('ensemble voting')
accuracies.append(accuracy)
precisions_w.append(precision_w)
recalls_w.append(recall_w)
f1_scores_w.append(f1_w)
precisions_m.append(precision_m)
recalls_m.append(recall_m)
f1_scores_m.append(f1_m)
metrics_df = pd.DataFrame({
    'Model Name': model_names,
    'Accuracy': accuracies,
    'Precision Weighted': precisions_w,
    'Recall Weighted': recalls_w,
    'F1-Score Weighted': f1_scores_w,
    'Precision Macro': precisions_m,
    'Recall Macro': recalls_m,
    'F1-Score Macro': f1_scores_m
})
metrics_df.sort_values(by=['Accuracy'], ascending=False, inplace=True)
metrics_df.to_csv("TFIDF.csv")

Training Models:

model Linear Regression trained
model Multinomial Naive Bayes trained
model Complement Naive Bayes trained
model KMeans trained


C:\Users\Bhavya\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


model XGB trained
model Support Vector Machine trained
model Decision Tree trained
model kNN trained
model SGD trained


c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


model adaboost trained
model Random Forest trained
Test Set Prediction:

Linear Regression
              precision    recall  f1-score   support

           0       0.62      0.12      0.20       429
           1       0.90      0.98      0.93      5757
           2       0.87      0.76      0.81      1249

    accuracy                           0.89      7435
   macro avg       0.80      0.62      0.65      7435
weighted avg       0.88      0.89      0.87      7435

Multinomial Naive Bayes
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       429
           1       0.79      1.00      0.88      5757
           2       0.95      0.08      0.15      1249

    accuracy                           0.79      7435
   macro avg       0.58      0.36      0.34      7435
weighted avg       0.77      0.79      0.71      7435

Complement Naive Bayes
              precision    recall  f1-score   support

           0       0.27      0.17      0.20  

c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

              precision    recall  f1-score   support

           0       0.50      0.20      0.29       429
           1       0.93      0.95      0.94      5757
           2       0.83      0.89      0.86      1249

    accuracy                           0.90      7435
   macro avg       0.75      0.68      0.70      7435
weighted avg       0.89      0.90      0.89      7435

Support Vector Machine
              precision    recall  f1-score   support

           0       0.52      0.08      0.13       429
           1       0.92      0.96      0.94      5757
           2       0.83      0.88      0.86      1249

    accuracy                           0.90      7435
   macro avg       0.76      0.64      0.64      7435
weighted avg       0.88      0.90      0.88      7435

Decision Tree
              precision    recall  f1-score   support

           0       0.35      0.26      0.30       429
           1       0.93      0.93      0.93      5757
           2       0.81      0.88     

c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Bhavya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

              precision    recall  f1-score   support

           0       0.62      0.06      0.11       429
           1       0.87      0.98      0.92      5757
           2       0.87      0.60      0.71      1249

    accuracy                           0.86      7435
   macro avg       0.79      0.55      0.58      7435
weighted avg       0.85      0.86      0.84      7435

--------------------VotingClassifier--------------------
              precision    recall  f1-score   support

           0    0.60331   0.17016   0.26545       429
           1    0.92041   0.96422   0.94181      5757
           2    0.84489   0.86789   0.85624      1249

    accuracy                        0.90222      7435
   macro avg    0.78954   0.66742   0.68783      7435
weighted avg    0.88943   0.90222   0.88841      7435



In [14]:
embeddings_index = {}
with open(r"C:\Users\Bhavya\Desktop\HateSpeechDataset\glove2.csv", "r", encoding="utf-8") as f:
    for line in f:
        values = line.split(' ')
        word = values[0] 
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
texts=df['cleaned_text4'].astype(str)
embeddings = []
n=0
ex=[]
for text in texts:
    text_embedding = []
    for word in text:
        if word in embeddings_index:
            text_embedding.append(embeddings_index[word])
    if len(text_embedding) > 0:
        text_embedding = np.mean(text_embedding, axis=0)
        embeddings.append(text_embedding)
    else:
        embeddings.append(np.zeros(300))
    n+=1
embeddings = np.array(embeddings)
len(embeddings)

y=df['class']
x_train,x_test,y_train,y_test=train_test_split(embeddings,y,test_size=0.3,stratify=y,random_state=51)
models = dict()
results=dict()
model_names,accuracies,precisions_w,recalls_w,f1_scores_w,precisions_m,recalls_m,f1_scores_m = [],[],[],[],[],[],[],[]
models['Linear Regression'] = LogisticRegression(max_iter=5000)
models['KMeans'] = KMeans(n_clusters=2, n_init=10, random_state=59)
models['XGB']= XGBClassifier(n_estimators=5000)
models['Support Vector Machine'] = SVC(kernel = 'sigmoid', gamma='scale',probability=True)
models['Decision Tree'] = DecisionTreeClassifier(max_depth=100)
models['kNN'] = KNeighborsClassifier()
models['SGD']=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=100, tol=None)
for i in models:
    models[i].fit(x_train,y_train)
    print('-'*20+i+'-'*20)
    y_pred=models[i].predict(x_test)
    if i!='SGD' and i!='KMeans':
        results[models[i]]=accuracy_score(y_test, y_pred)
    print(classification_report(y_test,y_pred,digits=5))
    model_names.append(i)
    accuracies.append(accuracy_score(y_test, y_pred))
    precisions_w.append(precision_score(y_test, y_pred,average='weighted'))
    recalls_w.append(recall_score(y_test, y_pred,average='weighted'))
    f1_scores_w.append(f1_score(y_test, y_pred,average='weighted'))
    precisions_m.append(precision_score(y_test, y_pred,average='macro'))
    recalls_m.append(recall_score(y_test, y_pred,average='macro'))
    f1_scores_m.append(f1_score(y_test, y_pred,average='macro'))
top_3_keys_tf = sorted(results, key=lambda k: results[k], reverse=True)[:3]
print("Top 3 keys with the highest values:")
print(top_3_keys_tf)
c1=top_3_keys_tf[0]
c2=top_3_keys_tf[1]
c3=top_3_keys_tf[2]
i='ensemble voting'
models['ensemble voting'] = VotingClassifier (estimators=[('clf1',c1), ('clf2',c2),('clf3',c3)], voting='soft')
models['ensemble voting'].fit(x_train, y_train)
y_pred = models['ensemble voting'].predict(x_test)
y_pred=models[i].predict(x_test)
print(classification_report(y_test,y_pred,digits=5))
model_names.append(i)
accuracies.append(accuracy_score(y_test, y_pred))
precisions_w.append(precision_score(y_test, y_pred,average='weighted'))
recalls_w.append(recall_score(y_test, y_pred,average='weighted'))
f1_scores_w.append(f1_score(y_test, y_pred,average='weighted'))
precisions_m.append(precision_score(y_test, y_pred,average='macro'))
recalls_m.append(recall_score(y_test, y_pred,average='macro'))
f1_scores_m.append(f1_score(y_test, y_pred,average='macro'))
metrics_df = pd.DataFrame({
'Model Name': model_names,
'Accuracy': accuracies,
'Precision Weighted': precisions_w,
'Recall Weighted': recalls_w,
'F1-Score Weighted': f1_scores_w,
'Precision Macro': precisions_m,
'Recall Macro': recalls_m,
'F1-Score Macro': f1_scores_m
})
metrics_df.sort_values(by=['Accuracy'], ascending=False, inplace=True)
metrics_df.to_csv("glove.csv", index=False)  

--------------------Linear Regression--------------------
              precision    recall  f1-score   support

           0    0.00000   0.00000   0.00000       103
           1    0.78407   1.00000   0.87897       374

    accuracy                        0.78407       477
   macro avg    0.39203   0.50000   0.43948       477
weighted avg    0.61476   0.78407   0.68917       477

--------------------KMeans--------------------
              precision    recall  f1-score   support

           0    0.21250   0.49515   0.29738       103
           1    0.78059   0.49465   0.60556       374

    accuracy                        0.49476       477
   macro avg    0.49655   0.49490   0.45147       477
weighted avg    0.65792   0.49476   0.53902       477



c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

--------------------XGB--------------------
              precision    recall  f1-score   support

           0    0.55556   0.14563   0.23077       103
           1    0.80444   0.96791   0.87864       374

    accuracy                        0.79036       477
   macro avg    0.68000   0.55677   0.55471       477
weighted avg    0.75070   0.79036   0.73874       477

--------------------Support Vector Machine--------------------
              precision    recall  f1-score   support

           0    0.00000   0.00000   0.00000       103
           1    0.78407   1.00000   0.87897       374

    accuracy                        0.78407       477
   macro avg    0.39203   0.50000   0.43948       477
weighted avg    0.61476   0.78407   0.68917       477



c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

--------------------Decision Tree--------------------
              precision    recall  f1-score   support

           0    0.26829   0.32039   0.29204       103
           1    0.80226   0.75936   0.78022       374

    accuracy                        0.66457       477
   macro avg    0.53528   0.53987   0.53613       477
weighted avg    0.68696   0.66457   0.67480       477

--------------------kNN--------------------
              precision    recall  f1-score   support

           0    0.29167   0.06796   0.11024       103
           1    0.78808   0.95455   0.86336       374

    accuracy                        0.76310       477
   macro avg    0.53987   0.51125   0.48680       477
weighted avg    0.68089   0.76310   0.70074       477

--------------------SGD--------------------
              precision    recall  f1-score   support

           0    0.00000   0.00000   0.00000       103
           1    0.78407   1.00000   0.87897       374

    accuracy                        0.78

...

--------------------Decision Tree--------------------
              precision    recall  f1-score   support

           0    0.35246   0.40952   0.37885       105
           1    0.78767   0.74434   0.76539       309

    accuracy                        0.65942       414
   macro avg    0.57007   0.57693   0.57212       414
weighted avg    0.67729   0.65942   0.66736       414

--------------------kNN--------------------
              precision    recall  f1-score   support

           0    0.47826   0.20952   0.29139       105
           1    0.77446   0.92233   0.84195       309

    accuracy                        0.74155       414
   macro avg    0.62636   0.56593   0.56667       414
weighted avg    0.69933   0.74155   0.70232       414

--------------------SGD--------------------
              precision    recall  f1-score   support

           0    0.00000   0.00000   0.00000       105
           1    0.74638   1.00000   0.85477       309

    accuracy                        0.74

c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

              precision    recall  f1-score   support

           0    0.50000   0.00952   0.01869       105
           1    0.74757   0.99676   0.85437       309

    accuracy                        0.74638       414
   macro avg    0.62379   0.50314   0.43653       414
weighted avg    0.68478   0.74638   0.64242       414

--------------------Linear Regression--------------------
              precision    recall  f1-score   support

           0    0.80804   0.91878   0.85986       197
           1    0.80723   0.60909   0.69430       110

    accuracy                        0.80782       307
   macro avg    0.80763   0.76394   0.77708       307
weighted avg    0.80775   0.80782   0.80054       307

--------------------KMeans--------------------
              precision    recall  f1-score   support

           0    0.65476   0.55838   0.60274       197
           1    0.37410   0.47273   0.41767       110

    accuracy                        0.52769       307
   macro avg    0.51443

c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

--------------------Decision Tree--------------------
              precision    recall  f1-score   support

           0    0.81407   0.82234   0.81818       197
           1    0.67593   0.66364   0.66972       110

    accuracy                        0.76547       307
   macro avg    0.74500   0.74299   0.74395       307
weighted avg    0.76457   0.76547   0.76499       307

--------------------kNN--------------------
              precision    recall  f1-score   support

           0    0.85795   0.76650   0.80965       197
           1    0.64885   0.77273   0.70539       110

    accuracy                        0.76873       307
   macro avg    0.75340   0.76961   0.75752       307
weighted avg    0.78303   0.76873   0.77230       307

--------------------SGD--------------------
              precision    recall  f1-score   support

           0    0.83673   0.83249   0.83461       197
           1    0.70270   0.70909   0.70588       110

    accuracy                        0.78